In [1]:
import json
import numpy as np
import pandas as pd
import cv2
import os
import tqdm
from scipy.io import loadmat
import matplotlib.pyplot as plt

In [2]:
# 원천 60개 한글 데이터
image_path = 'data/this_img/'
gt_path = 'data/this_label/'

In [15]:
image_path = 'data/korean_images/'
gt_path = 'data/korean_ground_truth/'

In [3]:
# 추가 한글 데이터
additional_image_path = 'data/signboard/additional_images/'
additional_gt_path = 'data/signboard/additional_labels/'

In [4]:
# 영어 데이터
en_image_path = 'data/signboard/en_images/'
en_gt_path = 'data/signboard/en_labels/'

In [3]:
# 원천 60개 한글 데이터
train_image_paths = []
train_gt_paths = []

In [6]:
# 추가 한글 데이터
additional_train_image_paths = []
additional_train_gt_paths = []

In [167]:
# 영어 데이터
en_train_image_paths = []
en_train_gt_paths = []

In [4]:
# 한글 이미지 데이터 (100개) 주소 리스트로 만들기

for new_file in tqdm.tqdm(os.listdir(gt_path)):

    name_split = new_file.split('.')
    image_name = name_split[0]     # "gt_" 뒤에가 이미지 이름
    image_name = image_name + '.jpg'

    if 'gt' in new_file:
        image_name = name_split[0]
        image_name = image_name + '.jpg'

    path_img = os.path.join(image_path , image_name)

    if not os.path.isfile(path_img):
        image_name = image_name.replace('.jpg', '.JPEG')
        path_img = os.path.join(image_path, image_name)

    train_image_paths.append(path_img)
    train_gt_paths.append(os.path.join(gt_path , new_file))


100%|██████████████████████████████████████| 213/213 [00:00<00:00, 16558.92it/s]


In [8]:
# 추가 한글 이미지 데이터 주소 리스트로 만들기

for new_file in tqdm.tqdm(os.listdir(additional_gt_path)):

    name_split = new_file.split('.')
    image_name = name_split[0]     # "gt_" 뒤에가 이미지 이름
    image_name = image_name + '.jpg'

    if 'gt' in new_file:
        image_name = name_split[0]
        image_name = image_name + '.jpg'

    path_img = os.path.join(additional_image_path , image_name)

    if not os.path.isfile(path_img):
        image_name = image_name.replace('.jpg', '.JPEG')
        path_img = os.path.join(additional_image_path, image_name)

    additional_train_image_paths.append(path_img)
    additional_train_gt_paths.append(os.path.join(additional_gt_path , new_file))



100%|████████████████████████████████████| 1401/1401 [00:00<00:00, 57969.18it/s]


In [149]:
# 영어 이미지 데이터 주소 리스트로 만들기

for new_file in tqdm.tqdm(os.listdir(en_gt_path)):
    
    name_split = new_file.split('.')
    image_name = name_split[0][3:]
    image_name = image_name + '.jpg'
    
    if 'gt' in new_file:
        image_name = name_split[0][3:]
        image_name = image_name + '.jpg'
    
    path_img = os.path.join(en_image_path , image_name)
    en_train_image_paths.append(path_img)
    en_train_gt_paths.append(os.path.join(en_gt_path , new_file))

100%|█████████████████████████████████████| 461/461 [00:00<00:00, 185813.39it/s]


In [5]:
X_final = []
Y_final = []
grid_h = 16
grid_w = 16
img_w = 512
img_h = 512

In [6]:
# 한글 데이터 (60개) 이미지 전처리

for z in tqdm.tqdm(range(len(train_image_paths))):
    
    new_file = train_image_paths[z]

    x = cv2.imread(train_image_paths[z])


    if x is not None:
        x_sl = 512/x.shape[1]
        y_sl = 512/x.shape[0]

        
        img = cv2.resize(x,(512,512))

        X_final.append(img)


        Y = np.zeros((grid_h,grid_w,1,5))

        file = train_gt_paths[z]

        with open(file, 'r') as file:
            data = json.load(file)
            data = data['annotations']


            for line in data:
                bb = line['bbox']
                #name = line['text']

                # x(중간좌표), y(중간좌표), width, height
                x = ((int((bb[0] + (bb[2]/2))) * x_sl) / img_w) * grid_w
                y = ((int((bb[1] + (bb[3]/2))) * y_sl) / img_h) * grid_h

                w = (int(bb[2]) * x_sl) / img_w
                h = (int(bb[3]) * y_sl) / img_h

                
                if x >= 16:
                    x = x-1
                if y >= 16:
                    y = y-1

                Y[int(y),int(x),0,0] = 1
                Y[int(y),int(x),0,1] = x - int(x)
                Y[int(y),int(x),0,2] = y - int(y)
                Y[int(y),int(x),0,3] = w
                Y[int(y),int(x),0,4] = h


            

            Y_final.append(Y)


100%|█████████████████████████████████████████| 213/213 [00:04<00:00, 52.13it/s]


In [11]:
# 추가 한글 데이터 이미지 전처리

for z in tqdm.tqdm(range(len(additional_train_image_paths))):
    
    new_file = additional_train_image_paths[z]

    x = cv2.imread(additional_train_image_paths[z])

    if x is not None:
        x_sl = 512/x.shape[1]
        y_sl = 512/x.shape[0]

        
        img = cv2.resize(x,(512,512))

        X_final.append(img)


        Y = np.zeros((grid_h,grid_w,1,5))

        file = additional_train_gt_paths[z]

        with open(file, 'r') as file:
            data = json.load(file)
            data = data['annotations']


            for line in data:
                bb = line['bbox']
                #name = line['text']

                # x(중간좌표), y(중간좌표), width, height
                x = ((int((bb[0] + (bb[2]/2))) * x_sl) / img_w) * grid_w
                y = ((int((bb[1] + (bb[3]/2))) * y_sl) / img_h) * grid_h

                w = (int(bb[2]) * x_sl) / img_w
                h = (int(bb[3]) * y_sl) / img_h

                
                if x >= 16:
                    x = x-1
                if y >= 16:
                    y = y-1

                Y[int(y),int(x),0,0] = 1
                Y[int(y),int(x),0,1] = x - int(x)
                Y[int(y),int(x),0,2] = y - int(y)
                Y[int(y),int(x),0,3] = w
                Y[int(y),int(x),0,4] = h


            

            Y_final.append(Y)

100%|███████████████████████████████████████| 1401/1401 [00:25<00:00, 55.48it/s]


In [153]:
# 영어 데이터 이미지 전처리

for z in tqdm.tqdm(range(len(en_train_image_paths))):
    
    new_file = en_train_image_paths[z]
    #print(new_file)
    
    x = cv2.imread(en_train_image_paths[z])
    
    if x is not None:
        x_sl = 512/x.shape[1]
        y_sl = 512/x.shape[0]
    
        img = cv2.resize(x,(512,512))
    
    
        X_final.append(img)
    

        i = " "

        if 'img' in new_file:
            i = ", "

        Y = np.zeros((grid_h,grid_w,1,5))

        file = en_train_gt_paths[z]
        name = open(file , 'r')
        data = name.read()
        
        data = data.split("\n")
        data = data[:-1]


        for li in data:
            #print(li)
            temp_list = []
            file_data = li.split(i)
            strr = file_data[4]
            bb = file_data[:4]


            xmin = int(bb[0])*x_sl
            xmax = int(bb[2])*x_sl
            ymin = int(bb[1])*y_sl
            ymax = int(bb[3])*y_sl

            #te = cv2.rectangle(img,(int(xmin),int(ymin)),(int(xmax),int(ymax)) , color = (0,255,0))

            w = (xmax - xmin)/img_w
            h = (ymax - ymin)/img_h

            x = ((xmax + xmin)/2)/img_w
            y = ((ymax + ymin)/2)/img_h
            x = x * grid_w
            y = y * grid_h

            Y[int(y),int(x),0,0] = 1
            Y[int(y),int(x),0,1] = x - int(x)
            Y[int(y),int(x),0,2] = y - int(y)
            Y[int(y),int(x),0,3] = w
            Y[int(y),int(x),0,4] = h
        #plt.imshow(te)
        #plt.show()

        Y_final.append(Y)

100%|█████████████████████████████████████████| 461/461 [00:07<00:00, 65.48it/s]


In [6]:
# 영어 데이터 이미지 전처리

for z in tqdm.tqdm(range(len(train_image_paths))):
    
    new_file = train_image_paths[z]
    #print(new_file)
    
    x = cv2.imread(train_image_paths[z])
    
    if x is not None:
        x_sl = 512/x.shape[1]
        y_sl = 512/x.shape[0]
    
        img = cv2.resize(x,(512,512))
    
    
        X_final.append(img)
    

        i = " "

        if 'img' in new_file:
            i = ", "

        Y = np.zeros((grid_h,grid_w,1,5))

        file = train_gt_paths[z]
        name = open(file , 'r')
        data = name.read()
        
        data = data.split("\n")
        data = data[:-1]


        for li in data:
            #print(li)
            temp_list = []
            file_data = li.split(i)
            strr = file_data[4]
            bb = file_data[:4]


            xmin = int(bb[0])*x_sl
            xmax = int(bb[2])*x_sl
            ymin = int(bb[1])*y_sl
            ymax = int(bb[3])*y_sl

            #te = cv2.rectangle(img,(int(xmin),int(ymin)),(int(xmax),int(ymax)) , color = (0,255,0))

            w = (xmax - xmin)/img_w
            h = (ymax - ymin)/img_h

            x = ((xmax + xmin)/2)/img_w
            y = ((ymax + ymin)/2)/img_h
            x = x * grid_w
            y = y * grid_h

            Y[int(y),int(x),0,0] = 1
            Y[int(y),int(x),0,1] = x - int(x)
            Y[int(y),int(x),0,2] = y - int(y)
            Y[int(y),int(x),0,3] = w
            Y[int(y),int(x),0,4] = h
        #plt.imshow(te)
        #plt.show()

        Y_final.append(Y)

100%|██████████████████████████████████████| 138/138 [00:00<00:00, 26310.92it/s]


In [7]:
print(len(X_final), len(Y_final))

212 212


In [8]:
X = np.array(X_final)
X_final = []

In [9]:
Y = np.array(Y_final)
Y_final = []

In [10]:
X = (X - 127.5)/127.5

In [11]:
print(X[0])

[[[ 0.43529412  0.35686275  0.36470588]
  [ 0.51372549  0.44313725  0.46666667]
  [ 0.57647059  0.50588235  0.52941176]
  ...
  [ 0.50588235  0.41960784  0.42745098]
  [ 0.37254902  0.27058824  0.27058824]
  [ 0.34901961  0.23921569  0.25490196]]

 [[ 0.58431373  0.48235294  0.49803922]
  [ 0.52941176  0.45882353  0.4745098 ]
  [ 0.69411765  0.62352941  0.64705882]
  ...
  [ 0.42745098  0.34117647  0.34117647]
  [ 0.45882353  0.36470588  0.36470588]
  [ 0.4745098   0.38039216  0.38039216]]

 [[ 0.37254902  0.27058824  0.29411765]
  [ 0.64705882  0.56078431  0.57647059]
  [ 0.55294118  0.49019608  0.49803922]
  ...
  [ 0.36470588  0.27058824  0.27058824]
  [ 0.38039216  0.29411765  0.30980392]
  [ 0.54509804  0.45882353  0.46666667]]

 ...

 [[ 0.4745098   0.57647059  0.55294118]
  [ 0.50588235  0.6         0.6       ]
  [ 0.3254902   0.41960784  0.41960784]
  ...
  [-0.52941176 -0.56078431 -0.83529412]
  [-0.52941176 -0.54509804 -0.81960784]
  [-0.49019608 -0.49803922 -0.80392157]]

 [

In [12]:
np.save('data/signboard/npy/final/X.npy', X)

In [13]:
np.save('data/signboard/npy/final/Y.npy', Y)

In [18]:
print(X[0])

[[[-0.59215686 -0.67058824 -0.70980392]
  [-0.57647059 -0.67058824 -0.68627451]
  [-0.55294118 -0.6627451  -0.65490196]
  ...
  [-0.61568627 -0.67843137 -0.85882353]
  [-0.55294118 -0.62352941 -0.81176471]
  [-0.49803922 -0.56862745 -0.78039216]]

 [[-0.92941176 -0.98431373 -1.        ]
  [-0.9372549  -0.99215686 -1.        ]
  [-0.92156863 -0.98431373 -0.98431373]
  ...
  [-0.40392157 -0.49803922 -0.71764706]
  [-0.23137255 -0.33333333 -0.55294118]
  [-0.18431373 -0.27058824 -0.52156863]]

 [[-0.00392157 -0.02745098 -0.05882353]
  [ 0.0745098   0.03529412  0.01176471]
  [ 0.16078431  0.12941176  0.12156863]
  ...
  [-0.20784314 -0.30980392 -0.56078431]
  [-0.14509804 -0.25490196 -0.51372549]
  [-0.18431373 -0.29411765 -0.57647059]]

 ...

 [[ 0.64705882  0.41176471  0.16862745]
  [ 0.63137255  0.38823529  0.18431373]
  [ 0.65490196  0.38823529  0.18431373]
  ...
  [-0.04313725 -0.29411765 -0.63921569]
  [-0.05882353 -0.30196078 -0.70196078]
  [-0.04313725 -0.28627451 -0.70196078]]

 [

In [14]:
print(Y[0])

[[[[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  ...

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  ...

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  ...

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]]


 ...


 [[[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  ...

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  ...

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]]


 [[[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  ...

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]

  [[0. 0. 0. 0. 0.]]]]
